In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import logging as lg
lg.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=lg.INFO)
import os
import pandas as pd
import numpy as np 
import pickle as pk
import glob
from fbprophet import Prophet
import sys
sys.path.append('../')
import wiki
from wiki import utils 
import multiprocessing as mp
total_proc = None
from tqdm import tqdm
from IPython.display import clear_output

# General column meanings
* y_org: what the orginal time series data is (should have missing values staying as missing!)
* y: what the model was trained on (the y in val section is kinda irrelevant then)
* yhat_org: original unchanged output of the models predict function
* yhat: final predictions including post processing
# Scoring
* Should all be scored using y_org vs yhat
* SHOULD MAKE SURE y_org HASN't HAD fillna(0)

# AIM: Get new smape -> v2.6
* series from pagedf into y_org

## Version 2.6
NOTE: may not be properly following full column meanings due to y_org being filled with 0s

Should set version directory name in next cell. Should describe version specifics (outliers, holidays, validation period)
* TRAINING
    * Val indexing on -60
    * Cut outliers out on upper 95% quartile `forecast.loc[forecast['yhat'] < 0,['yhat']] = 0.0`
    * Linear growth
    * Fill ALL other NaNs to 0 BUT KEEP y_org (get it from pagedf)
    * Now with try:except: for the `RuntimeError: k initialized to invalid value (-nan)` which replaces first `y` with 0.001
    
* PREDICTIONS
    * Truncating predictions at 0 
    * Rounding to nearest int

	* Need this now as the outlier cutting might remove the initial 0.001 we put in there
### Remarks
* ?

In [3]:
PROPHET_PATH = '../data/prophet/'
RESULTS_PATH = 'results/'

In [5]:
TARGET_VERSION = 'v2.6/'
BASE_VERSION = 'v2.5/'
#os.makedirs(PROPHET_PATH+TARGET_VERSION)

In [6]:
pagedf = pd.read_feather(PROPHET_PATH+'pagedf.f')

In [7]:
forecast_files = [x.split('/')[-1] for x in glob.glob(PROPHET_PATH+BASE_VERSION+'*df.f')]

In [8]:
forecast_files[18284]

'20274df.f'

In [18]:
pagedf[forecast_files[18284][:-4]].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: 20274, dtype: float64

In [10]:
[x[:-4] for x in forecast_files[:2]]

['61780', '88262']

In [11]:
v25df = pd.read_feather(PROPHET_PATH+BASE_VERSION+forecast_files[18284])

In [17]:
v25df.head()

,ds,t,trend,seasonal_lower,seasonal_upper,trend_lower,trend_upper,yhat_lower,yhat_upper,weekly,...,weekly_upper,yearly,yearly_lower,yearly_upper,seasonal,yhat,yhat_org,y,train,y_org
0,2015-07-01,0.000000,4.717248,-1.711822,-1.711822,4.717248,4.717248,0.119087,5.558690,-0.277588,...,-0.277588,-1.434234,-1.434234,-1.434234,-1.711822,3,3.005426,3.0,1.0,3.0
1,2015-07-02,0.002045,4.717247,-1.678052,-1.678052,4.717247,4.717247,0.450957,5.639848,-0.328540,...,-0.328540,-1.349512,-1.349512,-1.349512,-1.678052,3,3.039195,2.0,1.0,2.0
2,2015-07-03,0.004090,4.717246,-1.109673,-1.109673,4.717246,4.717246,0.898649,6.417234,0.147646,...,0.147646,-1.257319,-1.257319,-1.257319,-1.109673,4,3.607573,5.0,1.0,5.0
3,2015-07-04,0.006135,4.717244,-0.775480,-0.775480,4.717244,4.717244,0.943920,6.553741,0.383609,...,0.383609,-1.159089,-1.159089,-1.159089,-0.775480,4,3.941764,5.0,1.0,5.0
4,2015-07-05,0.008180,4.717243,-0.703721,-0.703721,4.717243,4.717243,1.351622,6.841876,0.352623,...,0.352623,-1.056344,-1.056344,-1.056344,-0.703721,4,4.013522,3.0,1.0,3.0


In [13]:
val_results = []
for file in tqdm(forecast_files):
    v25df = pd.read_feather(PROPHET_PATH+BASE_VERSION+file)
    # y_org to be the orginal timeseries data
    v25df.loc[:,'y_org'] = pagedf[file[:-4]]
    v25df.to_feather(PROPHET_PATH+TARGET_VERSION+file)
    full_smape = wiki.val.smape(v25df.y_org, v25df.yhat)
    val_smape = wiki.val.smape(v25df[v25df['train'] == 0].y_org,v25df[v25df['train'] == 0].yhat)
    val_results.append((file[:-4], full_smape, val_smape))
val_results = pd.DataFrame(val_results, columns=['page_index',TARGET_VERSION[:-1]+'_full',TARGET_VERSION[:-1]+'_val'])
val_results.to_feather(PROPHET_PATH+RESULTS_PATH+TARGET_VERSION[:-1]+'df.f')

100%|██████████| 145063/145063 [30:33<00:00, 79.10it/s]


In [14]:
resultsdf = pd.read_feather(PROPHET_PATH+RESULTS_PATH+TARGET_VERSION[:-1]+'df.f')

In [15]:
resultsdf['v2.6_val'].mean()
# base was v2.5 at 55.451

56.867810391849325

In [16]:
resultsdf['v2.6_val'].count()

143868